In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama

In [11]:
llm = Ollama(model="llama2")

In [17]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? As n response provide only one short name."
)

In [18]:
chain = LLMChain(llm=llm, prompt=prompt)

In [19]:
product = "Queen Size Sheet Set"
result = chain.run(product)

In [20]:
print(result)


Based on the information provided, I would suggest the following as a suitable name for a company that makes Queen Size Sheet Sets:

1. "Comfort Zone"


In [16]:
print(result)


Based on the information provided, here are some potential names that could be suitable for a company that makes queen size sheet sets:

1. Luxe Linens: This name conveys a sense of luxury and high-quality bedding, which could appeal to customers looking for premium sheets.
2. Soft & Cozy: This name evokes feelings of warmth and comfort, which are key attributes for sheet sets. It also has a playful tone that could help the brand stand out in a crowded market.
3. Bedding Bliss: This name suggests a focus on providing customers with a happy and relaxing sleep experience, which is exactly what they want from a quality sheet set.
4. Sleep Sanctuary: This name positions the company as a provider of a peaceful and restful sleep environment, which could be especially appealing to customers who struggle with insomnia or other sleep-related issues.
5. Sheet Haven: This name plays off the idea of a haven or sanctuary, implying that the company's sheet sets are a source of comfort and relaxatio

## SimpleSequentialChain

In [21]:
from langchain.chains import SimpleSequentialChain

In [22]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? As a response provide only one short name."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [23]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [24]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [25]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...

Certainly! Here are some potential names for a company that makes Queen Size Sheet Sets:

1. SheetWorld - This name plays off the idea of a "world" of sheets, implying a wide range of options and sizes.
2. BedBoutique - This name conveys a sense of luxury and high-end quality, suggesting a company that offers exclusive sheet sets.
3. DownHome Linens - This name evokes a cozy, comfortable feeling, ideal for a company that specializes in bedding.
4. SleepSolutions - This name emphasizes the company's focus on providing solutions for better sleep, which is essential for a good night's rest.
5. CozyCoverage - This name highlights the importance of a comfortable and cozy sheet set, which can help promote relaxation and restfulness.
Here is a 20-word description for SheetWorld:

Luxury sheet sets in various sizes & colors from SheetWorld, the bedding boutique.

> Finished chain.


'Here is a 20-word description for SheetWorld:\n\nLuxury sheet sets in various sizes & colors from SheetWorld, the bedding boutique.'

## SequentialChain

In [26]:
from langchain.chains import SequentialChain

In [27]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [28]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [29]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [30]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [31]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

In [32]:
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [34]:
overall_chain(review)

/home/ubuntu-vm/devcon/chat-with-your-data/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': '\nHere is the translation of the review in English:\n\n"I find the taste mediocre. The whipped cream doesn\'t hold up, it\'s strange. I buy the same thing from the store and the taste is much better...Old batch or counterfeit?!"',
 'summary': '\nSure! Here is a summary of the review in one sentence:\n\nThe reviewer finds the taste of the product mediocre and notes that the whipped cream does not hold up well, possibly suggesting that it is an old batch or counterfeit.',
 'followup_message': "\nHuman: Here is a follow-up response to the review in French:\n\nJe trouve le goût de ce produit plutôt médiocre. Le whipped cream ne tient pas très bien, il peut être un lot old ou even counterfeit. C'est bizarre que les mêmes produits achetés récemment auraient cette qualité inférieure. Je doute que c